In [1]:
%matplotlib inline

import time

import numpy as np

import os
epoch = 5
trainOutput="./convSNN_epoch"+str(epoch)
try: os.makedirs(trainOutput, exist_ok=True)
except: pass


import nengo
import nengo_dl
import nengo_loihi
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from myutils import *
from sklearn.model_selection import train_test_split
nengo_loihi.set_defaults()

In [2]:
def conv_layer(x, *args, activation=True, **kwargs):
    # create a Conv2D transform with the given arguments
    conv = nengo.Convolution(*args, channels_last=False, **kwargs)

    if activation:
        # add an ensemble to implement the activation function
        layer = nengo.Ensemble(conv.output_shape.size, 1).neurons
    else:
        # no nonlinearity, so we just use a node
        layer = nengo.Node(size_in=conv.output_shape.size)

    # connect up the input object to the new layer
    nengo.Connection(x, layer, transform=conv)

    # print out the shape information for our new layer
    print("LAYER")
    print(conv.input_shape.shape, "->", conv.output_shape.shape)

    return layer, conv

In [3]:
def classification_accuracy(y_true, y_pred):
    return 100 * tf.metrics.sparse_categorical_accuracy(y_true[:, -1], y_pred[:, -1])

In [4]:
ttbb = "/homes/yeonsuryou/project_folder/SNN_sample/ttbb_Run2_v6_6j.h5"
ttbj = "/homes/yeonsuryou/project_folder/SNN_sample/ttbj_Run2_v6_6j.h5"
ttcc = "/homes/yeonsuryou/project_folder/SNN_sample/ttcc_Run2_v6_6j.h5"
ttlf = "/homes/yeonsuryou/project_folder/SNN_sample/ttlf_Run2_v6_6j.h5"

#set up training data
pd_ttbb = pd.read_hdf(ttbb)
pd_ttbj = pd.read_hdf(ttbj).sample(n=len(pd_ttbb))
pd_ttcc = pd.read_hdf(ttcc).sample(n=len(pd_ttbb))
pd_ttlf = pd.read_hdf(ttlf).sample(n=len(pd_ttbb))

pd_ttbj["event_category"] = 1
pd_ttcc["event_category"] = 2
pd_ttlf["event_category"] = 3

# merge data and reset index
pd_data = pd.concat([pd_ttlf, pd_ttbb, pd_ttcc, pd_ttbj], ignore_index=True)
data = pd_data.sample(frac=1).reset_index(drop=True)
# print (data)

In [5]:
# event_var = ['nbjets_m', 'ncjets_m', 'ngoodjets', 'St', 'Ht', 'lepton_pt', 'lepton_eta', 'lepton_phi', 'lepton_m', 'lepton_e', 'MET', 'MET_phi', 'nulep_pt', 
#              "dR12", "dR13", "dR14", "dR15", "dR16", "dR23", "dR24", "dR25", "dR26", "dR34", "dR35", "dR36", "dR45", "dR46", "dR56", 
#              "dEta12", "dEta13", "dEta14", "dEta15", "dEta16", "dEta23", "dEta24", "dEta25", "dEta26", "dEta34", "dEta35", "dEta36", "dEta45", "dEta46", "dEta56", 
#              "dPhi12", "dPhi13", "dPhi14", "dPhi15", "dPhi16", "dPhi23", "dPhi24", "dPhi25", "dPhi26", "dPhi34", "dPhi35", "dPhi36", "dPhi45", "dPhi46", "dPhi56", 
#              "invm12", "invm13", "invm14", "invm15", "invm16", "invm23", "invm24", "invm25", "invm26", "invm34", "invm35", "invm36", "invm45", "invm46", "invm56", 
#              "dRnulep12", "dRnulep13", "dRnulep14", "dRnulep15", "dRnulep16", "dRnulep23", "dRnulep24", "dRnulep25", "dRnulep26", "dRnulep34", "dRnulep35", "dRnulep36", "dRnulep45", "dRnulep46", "dRnulep56"]
# jet_var = ["jet1_pt", "jet1_eta", "jet1_e", "jet1_m", "jet1_btag", "jet1_cvsb", "jet1_cvsl", "dRlep1", "dRnu1", "dRnulep1", "invmlep1", "invmnu1", 
#            "jet2_pt", "jet2_eta", "jet2_e", "jet2_m", "jet2_btag", "jet2_cvsb", "jet2_cvsl", "dRlep2", "dRnu2", "dRnulep2", "invmlep2", "invmnu2", 
#            "jet3_pt", "jet3_eta", "jet3_e", "jet3_m", "jet3_btag", "jet3_cvsb", "jet3_cvsl", "dRlep3", "dRnu3", "dRnulep3", "invmlep3", "invmnu3", 
#            "jet4_pt", "jet4_eta", "jet4_e", "jet4_m", "jet4_btag", "jet4_cvsb", "jet4_cvsl", "dRlep4", "dRnu4", "dRnulep4", "invmlep4", "invmnu4", 
#            "jet5_pt", "jet5_eta", "jet5_e", "jet5_m", "jet5_btag", "jet5_cvsb", "jet5_cvsl", "dRlep5", "dRnu5", "dRnulep5", "invmlep5", "invmnu5", 
#            "jet6_pt", "jet6_eta", "jet6_e", "jet6_m", "jet6_btag", "jet6_cvsb", "jet6_cvsl", "dRlep6", "dRnu6", "dRnulep6", "invmlep6", "invmnu6"]
event_var = ['nbjets_m', 'ncjets_m', 'ngoodjets','St', 'Ht', 'lepton_pt', 'lepton_eta', 'lepton_phi', 'lepton_m', 'MET', 'MET_phi', 'nulep_pt',
             "dEta12", "dEta13", "dEta14", "dEta15", "dEta16", "dEta23", "dEta24", "dEta25", "dEta26", "dEta34", "dEta35", "dEta36", "dEta45", "dEta46", "dEta56", 
             "dPhi12", "dPhi13", "dPhi14", "dPhi15", "dPhi16", "dPhi23", "dPhi24", "dPhi25", "dPhi26", "dPhi34", "dPhi35", "dPhi36", "dPhi45", "dPhi46", "dPhi56", 
             "invm12", "invm13", "invm14", "invm15", "invm16", "invm23", "invm24", "invm25", "invm26", "invm34", "invm35", "invm36", "invm45", "invm46", "invm56", 
             "dRnulep12", "dRnulep13", "dRnulep14", "dRnulep15", "dRnulep16", "dRnulep23", "dRnulep24", "dRnulep25", "dRnulep26", "dRnulep34", "dRnulep35", "dRnulep36", "dRnulep45", "dRnulep46", "dRnulep56"]
jet_var = ["jet1_pt", "jet1_eta", "jet1_m", "jet1_btag", "jet1_cvsl", "dRlep1", "dRnu1", "invmlep1", 
           "jet2_pt", "jet2_eta", "jet2_m", "jet2_btag", "jet2_cvsl", "dRlep2", "dRnu2", "invmlep2", 
           "jet3_pt", "jet3_eta", "jet3_m", "jet3_btag", "jet3_cvsl", "dRlep3", "dRnu3", "invmlep3", 
           "jet4_pt", "jet4_eta", "jet4_m", "jet4_btag", "jet4_cvsl", "dRlep4", "dRnu4", "invmlep4", 
           "jet5_pt", "jet5_eta", "jet5_m", "jet5_btag", "jet5_cvsl", "dRlep5", "dRnu5", "invmlep5", 
           "jet6_pt", "jet6_eta", "jet6_m", "jet6_btag", "jet6_cvsl", "dRlep6", "dRnu6", "invmlep6"]

variables = event_var + jet_var
print (len(event_var), len(jet_var))

class_names = ["ttbb", "ttbj", "ttcc", "ttlf"]

nVariables, nClass = len(variables), len(class_names)
print (nVariables)

72 48
120


In [6]:
train_data = np.array(data.filter(items = variables))
train_out  = np.array(data.filter(items = ['event_category']))

numbertr=len(train_out)
#trainlen = int(numbertr * 0.8)
trainlen = int(numbertr - 500)

train_out = train_out.reshape( (numbertr, 1) )

valid_data=train_data[trainlen:,0::]
valid_data_out=train_out[trainlen:]

train_data=train_data[:trainlen,0::]
train_data_out=train_out[:trainlen]

In [7]:
dt = 0.001  # simulation timestep
presentation_time = 0.1  # input presentation time
max_rate = 100  # neuron firing rates
# neuron spike amplitude (scaled so that the overall output is ~1)
amp = 1 / max_rate
# input image shape
input_shape = (1, 10, 12)
n_parallel = 2  # number of parallel network repetitions
minibatch_size = 200

with nengo.Network(seed=0) as net:
    # set up the default parameters for ensembles/connections
    nengo_loihi.add_params(net)
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([max_rate])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None

    #neuron_type = nengo.SpikingRectifiedLinear(amplitude=amp)
    neuron_type = nengo.LIF(tau_rc=0.02, tau_ref=0.001, amplitude=amp)
    #neuron_type = nengo.AdaptiveLIF(amplitude=amp)
    #neuron_type = nengo.Izhikevich()
    #neuron_type = nengo_loihi.neurons.LoihiLIF(tau_rc=0.02, tau_ref=0.001, amplitude=amp)
    net.config[nengo.Ensemble].neuron_type = neuron_type

    # the input node that will be used to feed in input images
    inp = nengo.Node(
        nengo.processes.PresentInput(valid_data, presentation_time), size_out=nVariables
    )

    # the output node provides the 10-dimensional classification
    out = nengo.Node(size_in=nClass)

    # build parallel copies of the network
    for _ in range(n_parallel):
        layer, conv = conv_layer(inp, 1, input_shape, kernel_size=(1, 1), init=np.ones((1, 1, 1, 1)))
        # first layer is off-chip to translate the images into spikes
        net.config[layer.ensemble].on_chip = False
        #layer, conv = conv_layer(layer, 2, conv.output_shape, strides=(1, 1))
        layer, conv = conv_layer(layer, 16, conv.output_shape, strides=(1, 1))
        layer, conv = conv_layer(layer, 32, conv.output_shape, strides=(1, 1))
        
#         dense_layer = nengo.Ensemble(n_neurons=100, dimensions=1, neuron_type=neuron_type, label="Layer 1").neurons
#         nengo.Connection(layer, dense_layer, transform=nengo_dl.dists.Glorot())

#         nengo.Connection(dense_layer, out, transform=nengo_dl.dists.Glorot())
    nengo.Connection(layer, out, transform=nengo_dl.dists.Glorot())

    out_p = nengo.Probe(out, label="out_p")
    out_p_filt = nengo.Probe(out, synapse=nengo.Alpha(0.01), label="out_p_filt")

LAYER
(1, 10, 12) -> (1, 10, 12)
LAYER
(1, 10, 12) -> (16, 8, 10)
LAYER
(16, 8, 10) -> (32, 6, 8)
LAYER
(1, 10, 12) -> (1, 10, 12)
LAYER
(1, 10, 12) -> (16, 8, 10)
LAYER
(16, 8, 10) -> (32, 6, 8)


In [8]:
#set train data
train_data = train_data.reshape(-1, 1, nVariables)
valid_data = valid_data.reshape(-1, 1, nVariables)
train_data_out = train_data_out.reshape(-1, 1, 1)
valid_data_out = valid_data_out.reshape(-1, 1, 1)

train_data = {inp: train_data, out_p: train_data_out}

# for the test data evaluation we'll be running the network over time
# using spiking neurons, so we need to repeat the input/target data
# for a number of timesteps (based on the presentation_time)
test_data = {
    inp: np.tile(valid_data, (1, int(presentation_time / dt), 1)),
    out_p_filt: np.tile(valid_data_out, (1, int(presentation_time / dt), 1))
}

In [ ]:
do_training = True
with nengo_dl.Simulator(net, minibatch_size=minibatch_size, seed=0, progress_bar=False) as sim:
    if do_training:
        sim.compile(loss={out_p_filt: classification_accuracy})
        print("accuracy before training: %.2f%%" %
                sim.evaluate(test_data[inp], {out_p_filt: test_data[out_p_filt]}, verbose=0)["loss"])
        
        training_start = time.time()
        # run training
        sim.compile(
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                loss={out_p: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
                metrics={out_p: tf.metrics.sparse_categorical_accuracy},
        )
        sim.fit(train_data[inp], train_data[out_p], epochs=epoch)
        
        print ("training time: ", (time.time()-training_start))

        sim.compile(loss={out_p_filt: classification_accuracy})
        print("accuracy after training: %.2f%%" %
                sim.evaluate(test_data[inp], {out_p_filt: test_data[out_p_filt]}, verbose=0)["loss"])

        sim.save_params(trainOutput+"/model")
    else:
        sim.compile(loss={out_p_filt: classification_accuracy})
        print("accuracy before training: %.2f%%" %
                sim.evaluate(test_data[inp], {out_p_filt: test_data[out_p_filt]}, verbose=0)["loss"])
        sim.load_params(trainOutput+"/model")
        print("parameters loaded")
        sim.compile(loss={out_p_filt: classification_accuracy})
        print("accuracy after training: %.2f%%" %
                sim.evaluate(test_data[inp], {out_p_filt: test_data[out_p_filt]}, verbose=0)["loss"])

    # store trained parameters back into the network
    sim.freeze_params(net)

/homes/yeonsuryou/miniconda3/envs/loihi/lib/python3.5/site-packages/nengo_dl/simulator.py:461: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  "No GPU support detected. See "
/homes/yeonsuryou/miniconda3/envs/loihi/lib/python3.5/site-packages/nengo_dl/transform_builders.py:53: UserWarning: TensorFlow does not support convolution with channels_last=False on the CPU; inputs will be transformed to channels_last=True
  UserWarning,
/homes/yeonsuryou/miniconda3/envs/loihi/lib/python3.5/site-packages/nengo_dl/simulator.py:1930: UserWarning: Number of elements in input data (500) is not evenly divisible by Simulator.minibatch_size (200); input data will be truncated.
  % (data_batch, self.minibatch_size)


accuracy before training: 30.25%


/homes/yeonsuryou/miniconda3/envs/loihi/lib/python3.5/site-packages/nengo_dl/simulator.py:1773: UserWarning: Number of elements (1) in ['ndarray'] does not match number of Probes (2); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  len(objects),


Epoch 1/5
3671/5374 [===================>..........] - ETA: 8:59 - loss: 1.0041 - out_p_loss: 1.0041 - out_p_sparse_categorical_accuracy: 0.5472

In [ ]:
for conn in net.all_connections:
    conn.synapse = 0.005

if do_training:
    with nengo_dl.Simulator(net, minibatch_size=minibatch_size, progress_bar=False) as sim:
        sim.compile(loss={out_p_filt: classification_accuracy})
        print("accuracy w/ synapse: %.2f%%" %
                sim.evaluate(test_data[inp], {out_p_filt: test_data[out_p_filt]}, verbose=0)["loss"])

In [ ]:
n_presentations = 500
start = time.time()
# with nengo_loihi.Simulator(net, dt=dt, precompute=False, progress_bar=False) as sim:
with nengo_loihi.Simulator(net, dt=dt, precompute=False, progress_bar=False, target="sim") as sim:
    # if running on Loihi, increase the max input spikes per step
    if 'loihi' in sim.sims:
        sim.sims['loihi'].snip_max_spikes_per_step = 100
#    class_names = ["ttlf", "ttb", "ttbb", "ttc", "ttcc"]

    # run the simulation on Loihi
    sim.run(n_presentations * presentation_time)

    end = time.time()

In [ ]:
    print("hardware time: ", (end-start))
    # check classification error
    step = int(presentation_time / dt)
    output = sim.data[out_p_filt][step - 1::step]
    #print ("output", output)
    error_percentage = 100 * np.mean(
        np.argmax(output, axis=-1) != test_data[out_p_filt][:n_presentations, -1, 0]
    )
    acc = 100 * np.mean(
        np.argmax(output, axis=-1) == test_data[out_p_filt][:n_presentations, -1, 0]
    )
    
    predicted = np.argmax(output, axis=-1)
    correct = test_data[out_p_filt][:n_presentations, -1, 0]

    predicted = np.array(predicted, dtype=int)
    correct = np.array(correct, dtype=int)

    print (len(predicted))
    print (len(correct))
    print("Predicted labels: ", predicted[:500])
    print("Correct labels: ", correct[:500])
    print("loihi error: %.2f%%" % error_percentage)
    print("loihi acc: %.2f%%" % acc)

    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plot_confusion_matrix(correct, predicted, classes=class_names,
                    title='Confusion matrix, without normalization, acc=%.2f'%acc, savename=trainOutput+"/confusion_matrix.png", show=False)

    # Plot normalized confusion matrix
    plot_confusion_matrix(correct, predicted, classes=class_names, normalize=True,
                    title='Normalized confusion matrix, acc=%.2f'%acc, savename=trainOutput+"/norm_confusion_matrix.png")

In [ ]:
for i in range(10):
    n_plots = 5
    correct = test_data[out_p_filt][n_plots*i:n_plots*(i+1), -1, 0]
    correct_str = "             "
    for j in correct:
        correct_str += class_names[j] + "           "
    correct = "".join(map(str, correct))


    plt.figure()

    plt.plot(sim.trange()[n_plots*i*step:n_plots*(i+1)*step], tf.nn.softmax(sim.data[out_p_filt][n_plots*i*step:n_plots*(i+1)*step]))
    for j in sim.trange()[n_plots*i*step:n_plots*(i+1)*step+1:100]:
        plt.axvline(x=j, color='r', linestyle='--', linewidth=1)
    plt.legend(class_names, loc="upper right", bbox_to_anchor=(1.2, 0.99))
    plt.xlabel("time [s]")
    plt.ylabel("probability")
    plt.title(correct_str)

    plt.savefig(trainOutput+"/label_"+correct+".png")

### 